In [ ]:
# !pip install tensorflow numpy librosa matplotlib

^C


In [ ]:
#pip install tensorflow keras numpy pandas scikit-learn nltk

In [2]:
import pandas as pd

# Load the datasets
df_train = pd.read_csv('/content/twitter_training.csv', names=['label', 'text'])
df_val = pd.read_csv('/content/twitter_validation.csv', names=['label', 'text'])

# Display the first few rows of each dataframe
print("Training DataFrame:")
print(df_train.head())

print("\nValidation DataFrame:")
print(df_val.head())


Training DataFrame:
                     label                                               text
2401 Borderlands  Positive  im getting on borderlands and i will murder yo...
     Borderlands  Positive  I am coming to the borders and I will kill you...
     Borderlands  Positive  im getting on borderlands and i will kill you ...
     Borderlands  Positive  im coming on borderlands and i will murder you...
     Borderlands  Positive  im getting on borderlands 2 and i will murder ...

Validation DataFrame:
                     label                                               text
3364 Facebook   Irrelevant  I mentioned on Facebook that I was struggling ...
352  Amazon        Neutral  BBC News - Amazon boss Jeff Bezos rejects clai...
8312 Microsoft    Negative  @Microsoft Why do I pay for WORD when it funct...
4371 CS-GO        Negative  CSGO matchmaking is so full of closet hacking,...
4433 Google        Neutral  Now the President is slapping Americans in the...


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import re
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Text cleaning and preprocessing
def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\@\w+|\#', '', text)
        text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    else:
        text = ""  # Replace non-string values with an empty string
    return text

# Apply cleaning to both training and validation dataframes
df_train['cleaned_text'] = df_train['text'].apply(clean_text)
df_val['cleaned_text'] = df_val['text'].apply(clean_text)

# Keep track of indices during the split
df_train_with_index = df_train.copy()
df_train_with_index.reset_index(drop=True, inplace=True)

# Split data while retaining indices
X_train_text, X_test_text, y_train, y_test, idx_train, idx_test = train_test_split(
    df_train_with_index['cleaned_text'], df_train_with_index['label'], df_train_with_index.index, test_size=0.2, random_state=42
)

# Tokenization and padding (fitting on training data and transforming both)
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train_text)

X_train = tokenizer.texts_to_sequences(X_train_text)
X_train = pad_sequences(X_train, maxlen=100)

X_test = tokenizer.texts_to_sequences(X_test_text)
X_test = pad_sequences(X_test, maxlen=100)

# Convert labels to categorical for both datasets
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

# The resulting X_train, X_test, y_train, and y_test are ready for model training and evaluation


In [5]:
num_classes_train = y_train.shape[1]
num_classes_val = y_test.shape[1]

print(f"Number of classes in training set: {num_classes_train}")
print(f"Number of classes in validation set: {num_classes_val}")


Number of classes in training set: 4
Number of classes in validation set: 4


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [6]:
# Building the CNN + LSTM model
model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=20000, output_dim=128, input_length=100))

# CNN layers
model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM layer
model.add(LSTM(100, return_sequences=True))
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer: Assuming four classes (e.g., positive, negative, neutral, and one more)
model.add(Dense(4, activation='softmax'))  # Updated to 4 units

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=2)

# Evaluate on the test set
score, acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {acc}')


Epoch 1/10
747/747 - 107s - 144ms/step - accuracy: 0.5533 - loss: 1.0519 - val_accuracy: 0.6998 - val_loss: 0.7825
Epoch 2/10
747/747 - 142s - 190ms/step - accuracy: 0.8007 - loss: 0.5583 - val_accuracy: 0.7926 - val_loss: 0.5596
Epoch 3/10
747/747 - 102s - 137ms/step - accuracy: 0.8758 - loss: 0.3475 - val_accuracy: 0.8183 - val_loss: 0.5306
Epoch 4/10
747/747 - 146s - 196ms/step - accuracy: 0.9095 - loss: 0.2449 - val_accuracy: 0.8236 - val_loss: 0.5417
Epoch 5/10
747/747 - 140s - 188ms/step - accuracy: 0.9305 - loss: 0.1846 - val_accuracy: 0.8237 - val_loss: 0.5943
Epoch 6/10
747/747 - 108s - 144ms/step - accuracy: 0.9410 - loss: 0.1553 - val_accuracy: 0.8274 - val_loss: 0.6293
Epoch 7/10
747/747 - 149s - 200ms/step - accuracy: 0.9495 - loss: 0.1275 - val_accuracy: 0.8385 - val_loss: 0.6336
Epoch 8/10
747/747 - 130s - 173ms/step - accuracy: 0.9537 - loss: 0.1160 - val_accuracy: 0.8348 - val_loss: 0.7089
Epoch 9/10
747/747 - 145s - 194ms/step - accuracy: 0.9568 - loss: 0.1077 - val_a

In [8]:
from sklearn.metrics import classification_report

# Convert the one-hot encoded predictions and true labels to class indices
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)

# Print the classification report
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive', 'Irrelevant']))


467/467 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step
              precision    recall  f1-score   support

    Negative       0.84      0.80      0.82      2592
     Neutral       0.87      0.87      0.87      4519
    Positive       0.79      0.84      0.82      3596
  Irrelevant       0.86      0.85      0.86      4230

    accuracy                           0.84     14937
   macro avg       0.84      0.84      0.84     14937
weighted avg       0.84      0.84      0.84     14937



In [11]:
def predict_classes(sample_text):
    cleaned_sample_text = clean_text(sample_text)

    # Tokenize and pad the text
    sample_sequence = tokenizer.texts_to_sequences([cleaned_sample_text])
    sample_padded = pad_sequences(sample_sequence, maxlen=100)
    # Predict the class using the model
    sample_prediction = model.predict(sample_padded)

    predicted_class = np.argmax(sample_prediction, axis=1)[0]
    class_names = ['Negative', 'Neutral', 'Positive', 'Irrelevant']
    print(f"Sample Text: {sample_text}")
    print(f"Predicted Class: {class_names[predicted_class]}")



def predict_sarcasm(text):
    if not isinstance(text, str):
        print("Text entered is not a string")
        return

    # Normalize text
    text = text.lower().strip()

    # sarcasm-related patterns
    sarcasm_patterns = [
        r'\byeah,? right\b',
        r'\btotally\b',
        r'\bsure\b',
        r'\bof course\b',
        r'\bas if\b',
        r'\bgreat,? just what i needed\b',
        r'\blove that for me\b',
        r'\bwhat a surprise\b',
        r'\bthanks a lot\b',
    ]

    # Punctuation or formatting cues
    punctuation_patterns = [
        r'!{2,}',  # Multiple exclamation marks
        r'\.{3,}',  # Ellipses
        r'\b(not|never|no way) (really|totally|at all)\b',  # Contrasting sentiments
    ]

    # Check sarcasm patterns
    for pattern in sarcasm_patterns + punctuation_patterns:
        if re.search(pattern, text):
            print("Sarcastic")

    print("Not Sarcastic")
    return

In [12]:
sample_text = "I absolutely love this product, it is fantastic!"
predict_classes(sample_text)
predict_sarcasm(sample_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Sample Text: I absolutely love this product, it is fantastic!
Predicted Class: Irrelevant
Not Sarcastic


In [13]:
!pip install streamlit pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.7 MB/s eta 0:00:00


In [27]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [14]:
model.save('/content/model.h5')

In [30]:
%%writefile app.py
import streamlit as st
import re
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


with open('/content/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
model = load_model("/content/model.h5")

# Cleaning Function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Functions
def predict_classes(sample_text):
    cleaned_sample_text = clean_text(sample_text)

    # Tokenize and pad the text
    sample_sequence = tokenizer.texts_to_sequences([cleaned_sample_text])
    sample_padded = pad_sequences(sample_sequence, maxlen=100)

    # Predict the class using the model
    sample_prediction = model.predict(sample_padded)
    predicted_class = np.argmax(sample_prediction, axis=1)[0]

    class_names = ['Negative', 'Neutral', 'Positive', 'Irrelevant']
    return class_names[predicted_class]

def predict_sarcasm(text):
    if not isinstance(text, str):
        return "Text entered is not a string"

    # Normalize text
    text = text.lower().strip()

    # sarcasm-related patterns
    sarcasm_patterns = [
        r'\byeah,? right\b',
        r'\btotally\b',
        r'\bsure\b',
        r'\bof course\b',
        r'\bas if\b',
        r'\bgreat,? just what i needed\b',
        r'\blove that for me\b',
        r'\bwhat a surprise\b',
        r'\bthanks a lot\b',
    ]

    # Punctuation or formatting cues
    punctuation_patterns = [
        r'!{2,}',  # Multiple exclamation marks
        r'\.{3,}',  # Ellipses
        r'\b(not|never|no way) (really|totally|at all)\b',  # Contrasting sentiments
    ]

    # Check sarcasm patterns
    for pattern in sarcasm_patterns + punctuation_patterns:
        if re.search(pattern, text):
            return "Sarcastic"

    return "Not Sarcastic"

# Streamlit App
st.title("Text Analysis: Sentiment and Sarcasm Detection")
st.write("Enter your text below, and the app will predict its sentiment class and sarcasm.")

# Input Text
user_input = st.text_area("Input Text", value="", placeholder="Type your text here...")

if st.button("Analyze"):
    if user_input.strip():
        # Predict Sentiment Class
        sentiment_result = predict_classes(user_input)

        # Predict Sarcasm
        sarcasm_result = predict_sarcasm(user_input)

        # Display Results
        st.write(f"**Sentiment Class:** {sentiment_result}")
        st.write(f"**Sarcasm Detection:** {sarcasm_result}")
    else:
        st.warning("Please enter some text to analyze.")

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

In [ ]:
!tensorflowjs_converter --input_format keras /content/model.h5 /content/web_model


In [ ]:
!ls /content/web_model

In [ ]:
# Navigate to the directory
%cd /content/index.html

# Start the HTTP server
!python3 -m http.server
